In [10]:
""" Live Trainding with Python and Binance

Step 4) Get data from DB, apply Strategy and Excecute Order
Step 5) Order Execution 

"""

' Live Trainding with Python and Binance\n\nStep 4) Get data from DB, apply Strategy and Excecute Order\nStep 5) Order Execution \n\n'

In [5]:
import pandas as pd 
from binance.client import Client
import sqlalchemy

%run Keys.py
client = Client(api_key,secret_key)


In [6]:
# Step 4 : Get data from Database

engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')
df = pd.read_sql('BTCUSDT', engine)

In [ ]:
"""
    Trendfollowing
    If the crypto was rising by z% -> Buy
    exit whe progit is above 0.15% or loss is crossing -0.15%
"""

In [7]:
def strategy(entry, lookback, qty, open_position=False):
    while   True:
        df = pd.read_sql('BTCUSDT', engine)
        lookbackperiod = df.iloc[-lookback:]
        cumret = (lookbackperiod.Price.pct_change()+1).cumprod()-1

        if not open_position:
            if cumret[cumret.last_valid_index()] > entry :
                #order = client.create_order(symbol='BTCUSDT', side='Buy', type= 'MARKET',quatity=qty)
                df = pd.read_sql('BTCUSDT', engine)
                time_order= df.Time[df.last_valid_index()]
                print("order buy")
                open_position = True
                break
    if open_position:
            while True:
                df = pd.read_sql('BTCUSDT', engine)
                #sincebuy =df.loc[df.Time > pd.to_datetime(order['transactTime'],units='ms')]
                sincebuy =df.loc[df.Time > pd.to_datetime(time_order,units='ms')]
                if len(sincebuy)>1: #* Es necesario q pase al menos un segundo para decidir si vender o esperar
                    sincebuyret = (sincebuy.Price.pct_change()+1).cumprod()-1
                    
                    last_entry = sincebuyret[sincebuyret.last_valid_index()]
                    if last_entry > 0.0015 or last_entry < -0.0015:
                        #order = client.create_order(symbol='BTCUSDT', side='Sell', type= 'MARKET',quatity=qty)
                        print("order sell")
                        break

In [8]:
strategy(0.001,60,0.001)